In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

## Scrap Contents for the required URLs
City,country,population,coordination

In [3]:
cities = [
            "London"
            ,"Paris"
            ,"Berlin"
            ,"Moscow"
            ,"Istanbul"
            ,"Barcelona"
            ,'Saint Petersburg'
            ,'Madrid'
            ,'Kyiv'
            ,'Rome'
            ,'Bucharest'
            ,'Minsk'
            ,'Vienna']
soups=[]
url_wiki = "https://en.wikipedia.org/wiki/"
headers = {'Accept-Language': 'en-US,en;q=0.8'}

for city in cities:
    response = requests.get(url_wiki+city, headers = headers)
    if response.status_code == 200:
        soups.append(BeautifulSoup(response.content, "html.parser"))
    else:
        print(f"city {city} doesn't have a wiki")
        soups.append("")

In [4]:
country = []
coordinate = []
pop_des = []
pop_value = []

In [5]:
for i,city in enumerate(cities):
    table_name = "table.infobox.ib-settlement.vcard > tbody "
    for trs in soups[i].select(table_name + " tr"):
        #Find Country
        if trs and ( "Country" in trs.get_text()):
           country.append(trs.findNext('td').get_text().strip())
            
        # Find Coordinate
        elif trs and ( "Coordinates" in trs.get_text()):
            coordinate.append(trs.findNext('span').get_text().split("/")[1].strip().replace(u'\ufeff', ''))
        # Find Population
        elif trs and ( "Population" in trs.get_text()):
            #description
            pop_des.append(trs.findNext('th').get_text().split("[")[0].replace(u'\xa0', ' '))
            #population value
            x=trs.findNext('td').get_text()
            x=x.replace(",","").split("[")[0]
            try:
                x= int(x)
                pop_value.append(x)
            except:
                pop_value.append("NotAvailable")
            break

In [6]:
len(cities)==len(country) == len(coordinate) == len(pop_des) == len(pop_value)

True

In [7]:
cities_df = pd.DataFrame(
    {"cities": cities,
     "country": country,
     "coordinate": coordinate,
     "pop_des": pop_des,
     "pop_value":pop_value
    }
)

## Export Final df

In [8]:
cities_df

,cities,country,coordinate,pop_des,pop_value
0,London,England,51.50722°N 0.12750°W,Population (2021 except where stated),8799800
1,Paris,France,48.856613°N 2.352222°E,Population (Jan. 2019),2165423
2,Berlin,Germany,52.52000°N 13.40500°E,Population (2021),3677472
3,Moscow,Russia,55.75583°N 37.61722°E,Population (2021 Census),13010112
4,Istanbul,Turkey,41.01361°N 28.95500°E,Population (31 December 2021),15840900
5,Barcelona,Spain,41.383°N 2.183°E,Population (2018),1620343
6,Saint Petersburg,Russia,59.93750°N 30.30861°E,Population,5351935
7,Madrid,Spain,40.41667°N 3.70250°W,Population (2018),3223334
8,Kyiv,Ukraine,50.45000°N 30.52333°E,Population (1 January 2021),2962180
9,Rome,Italy[a],41.89333°N 12.48278°E,Population (31 December 2019),NotAvailable


In [9]:
cities_df.to_pickle("cities2.pkl")

## Get Weather from API

In [73]:
import requests
import json
from api_key import API_key

In [74]:
def ConvertCoord(coord):
    #convert lat and long
    lat,long = coord.replace("°E","").replace("°N","").split(" ")
    if lat.find("°S") != -1 :
        lat="-" + lat.replace("°S","")
    if long.find("°W") != -1 :
        long="-" + long.replace("°W","")
    try:
        lat= float(lat)
        long= float(long)
    except:
        print("ERROR, COORDINATES are not Convertable")
    return (lat,long)

In [72]:
# coor= "52.52000°N 13.40500°E"  
# lat,long = ConvertCoord(coor)
# print(lat,long)
# url_weather= f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={long}&appid={API_key}"
# weather_response = requests.get(url_weather)
# weather_json_l= weather_response.json()
# #print(json.dumps(weather_json_l, indent=2))

52.52 13.405


In [75]:
weather_json_l=[]
for coor in cities_df.coordinate:
    lat,long = ConvertCoord(coor)
    url_weather= f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={long}&appid={API_key}&units=metric"
    weather_response = requests.get(url_weather)
    if weather_response.status_code != 200:
        print(weather_response.text)
        weather_json = ""
    else:
        weather_json_l.append(weather_response.json())
       

In [ ]:
# from pprint import pprint
# pprint(weather_json_l[0])

In [76]:
cities_weath = {"cities":[]
                ,"forcast_time":[]
                ,"outlook":[]
                ,"temperature":[]
                ,"feels_like":[]
                ,"wind_speed":[]
                ,"rain_prob":[]
                }

for j,city_json in enumerate(weather_json_l):
    current_city = cities_df.loc[j].cities
    for time in city_json["list"]:
        cities_weath["cities"].append(current_city)
        cities_weath["forcast_time"].append(time["dt_txt"])
        cities_weath["temperature"].append(time["main"]["temp"])
        cities_weath["outlook"].append(time["weather"][0]["description"])
        cities_weath["feels_like"].append(time["main"]["feels_like"])
        cities_weath["wind_speed"].append(time["wind"]["speed"])
        cities_weath["rain_prob"].append(time["pop"])
        

In [77]:
cities_weather_df = pd.DataFrame(cities_weath)

### Final df

In [78]:
cities_weather_df

,cities,forcast_time,outlook,temperature,feels_like,wind_speed,rain_prob
0,London,2023-01-03 18:00:00,light rain,10.76,10.24,8.59,1.0
1,London,2023-01-03 21:00:00,light rain,11.32,10.88,8.08,1.0
2,London,2023-01-04 00:00:00,light rain,12.58,12.14,8.37,1.0
3,London,2023-01-04 03:00:00,overcast clouds,12.43,11.90,8.47,0.0
4,London,2023-01-04 06:00:00,overcast clouds,12.07,11.60,8.08,0.0
...,...,...,...,...,...,...,...
515,Vienna,2023-01-08 03:00:00,overcast clouds,4.52,3.44,1.47,0.0
516,Vienna,2023-01-08 06:00:00,overcast clouds,3.93,2.32,1.82,0.0
517,Vienna,2023-01-08 09:00:00,overcast clouds,5.16,2.48,3.32,0.0
518,Vienna,2023-01-08 12:00:00,overcast clouds,7.83,4.31,6.51,0.0
